# Composability

This notebook shows off some basic functionality around LangChain Expression Language, which makes it really easy to compose arbitrary chains.

For a much deeper dive, see:

- [Full LangChain Expression Language Documentation](https://python.langchain.com/docs/expression_language/)

## Basic Composability

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

In [3]:
chain = prompt | model | output_parser

In [4]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

### Batching

In [5]:
chain.batch([{"topic": "bear"}, {"topic": "clowns"}])

["Why don't bears like fast food?\nBecause they can't catch it!",
 "Why don't clowns ever get angry? Because they always keep their cool!"]

### Streaming

In [6]:
for s in chain.stream({"topic": "bears"}):
    print(s)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



## RunnablePassthrough

In [7]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

In [8]:
chain = prompt | model | output_parser

In [9]:
question = "what is langsmith"
context = "langsmith is a testing and observability platform built by the langchain team"
chain.invoke({"question": question, "context": context})

'Langsmith is a testing and observability platform developed by the Langchain team.'

In [10]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain

In [11]:
retrieval_chain.invoke({"question": "what is langsmith"})

'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents. It allows users to debug, test, evaluate, and monitor chains and intelligent agents built on any Language Model (LLM) framework. LangSmith seamlessly integrates with LangChain, an open source framework for building with LLMs.'

## RunnableParallel

In [12]:
from langchain_core.runnables import RunnableParallel

In [13]:
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser

In [14]:
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})

In [15]:
parallel_chain.invoke({"question": "what is langsmith"})

{'retrieved_answer': 'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents. It allows users to debug, test, evaluate, and monitor chains and intelligent agents built on any Language Model (LLM) framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs.',
 'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It does not appear to be a widely known term or concept.'}

In [16]:
for s in parallel_chain.stream({"question": "what is langsmith"}):
    print(s)

{'simple_answer': ''}
{'simple_answer': 'I'}
{'simple_answer': "'m"}
{'simple_answer': ' sorry'}
{'simple_answer': ','}
{'simple_answer': ' but'}
{'simple_answer': ' I'}
{'simple_answer': ' couldn'}
{'simple_answer': "'t"}
{'simple_answer': ' find'}
{'simple_answer': ' any'}
{'simple_answer': ' information'}
{'simple_answer': ' on'}
{'simple_answer': ' "'}
{'simple_answer': 'lang'}
{'simple_answer': 'smith'}
{'simple_answer': '."'}
{'simple_answer': ' It'}
{'simple_answer': ' is'}
{'simple_answer': ' possible'}
{'simple_answer': ' that'}
{'simple_answer': ' it'}
{'simple_answer': ' may'}
{'simple_answer': ' be'}
{'simple_answer': ' a'}
{'simple_answer': ' lesser'}
{'simple_answer': '-known'}
{'simple_answer': ' term'}
{'simple_answer': ' or'}
{'simple_answer': ' a'}
{'simple_answer': ' specific'}
{'simple_answer': ' reference'}
{'simple_answer': ' that'}
{'simple_answer': ' requires'}
{'simple_answer': ' more'}
{'simple_answer': ' context'}
{'simple_answer': '.'}
{'simple_answer': ' Co

In [17]:
result = {}
for s in parallel_chain.stream({"question": "what is langsmith"}):
    for k,v in s.items():
        if k not in result:
            result[k] = ""
        result[k] += v
    print(result)

{'simple_answer': ''}
{'simple_answer': 'I'}
{'simple_answer': "I'm"}
{'simple_answer': "I'm sorry"}
{'simple_answer': "I'm sorry,"}
{'simple_answer': "I'm sorry, but"}
{'simple_answer': "I'm sorry, but I"}
{'simple_answer': "I'm sorry, but I couldn"}
{'simple_answer': "I'm sorry, but I couldn't"}
{'simple_answer': "I'm sorry, but I couldn't find"}
{'simple_answer': "I'm sorry, but I couldn't find any"}
{'simple_answer': "I'm sorry, but I couldn't find any information"}
{'simple_answer': "I'm sorry, but I couldn't find any information on"}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Lang'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith."'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith." It'}
{'simple_answer': 'I\'m sorry, but I coul

{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It seems to be a term that is not widely recognized or used. Could you please provide more context or clarify your question?', 'retrieved_answer': 'LangSmith is a platform that offers features for debugging, testing, evaluating, and monitoring Language Learning Models (LLMs) and AI applications. It helps users trace and evaluate their language model applications and intelligent agents, providing visibility'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It seems to be a term that is not widely recognized or used. Could you please provide more context or clarify your question?', 'retrieved_answer': 'LangSmith is a platform that offers features for debugging, testing, evaluating, and monitoring Language Learning Models (LLMs) and AI applications. It helps users trace and evaluate their language model applications and intelligent agents, providing visibility into'}
{'s